<a href="https://colab.research.google.com/github/vinnik-dmitry07/Chatbot/blob/main/train_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install --quiet parlai

Thu Feb 18 17:11:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from pathlib import Path

GDRIVE_ROOT = Path('/content/drive/MyDrive/')
SAVE_DIR = GDRIVE_ROOT / 'chatbot_model'
DATA_DIR = GDRIVE_ROOT / 'chatbot_data'

In [ ]:
from datetime import timedelta

EPISODE_DT = timedelta(minutes=3)  # change to split messages in separate dialogues if time delta is greater than EPISODE_DT
TRAIN_PART, TEST_PART, VALID_PART = 0.996, 0.002, 0.002

assert TRAIN_PART + TEST_PART + VALID_PART == 1

In [ ]:
from google.colab import drive

drive.mount(str(GDRIVE_ROOT.parent))

Mounted at /content/drive


In [ ]:
import json

with open(DATA_DIR / 'result.json', 'r', encoding='utf8') as f:
    raw_messages = json.load(f)['messages']

In [ ]:
from datetime import datetime

filtered_messages = []
for msg in raw_messages:
    if (
            'from' in msg and
            'from_id' in msg and
            'mime_type' not in msg and
            msg['text'] and
            isinstance(msg['text'], str) and
            len(msg['text']) < 50
    ):
        msg1 = msg.copy()
        msg1['date'] = datetime.strptime(msg1['date'], '%Y-%m-%dT%H:%M:%S')
        filtered_messages.append(msg1)

In [ ]:
import re

joined_messages = []
for i in range(len(filtered_messages)):
    alphanum_text = re.sub(r'[^A-Za-z0-9 ]+', '', filtered_messages[i]['text']).strip()
    if alphanum_text:
        if (    
                joined_messages and    
                filtered_messages[i - 1]['from_id'] == filtered_messages[i]['from_id'] and
                filtered_messages[i - 1]['date'] - filtered_messages[i]['date'] <= EPISODE_DT
        ):
            joined_messages[-1]['text'] += ' ' + alphanum_text
        else:
            new_message = filtered_messages[i].copy()
            new_message['text'] = alphanum_text
            joined_messages.append(new_message)

In [ ]:
def partition(alist, indices):
    return [alist[a:b] for a, b in zip([0] + indices, indices + [None])]

In [ ]:
def save_jsonl(messages, suffix, human_readable=False):
    time_diffs = [messages[i + 1]['date'] - messages[i]['date'] for i in range(len(messages) - 1)]
    split_positions = [i + 1 for i in range(len(time_diffs)) if time_diffs[i] > EPISODE_DT]
    episodes = partition(messages, split_positions)
    print(f'{suffix} episodes: {len(episodes)}, messages: {len(messages)}')

    with open(DATA_DIR / f'data_{suffix}.jsonl', 'w', **({'encoding': 'utf8'} if human_readable else {})) as outfile:
        for episode in episodes:
            dialog = [{'id': i % 2, 'text': msg['text']} for i, msg in enumerate(episode)]
            episode = {'dialog': [dialog]}
            json.dump(episode, outfile, **({'ensure_ascii': False} if human_readable else {}))
            outfile.write('\n')

In [ ]:
import numpy as np

train, test, valid = np.split(joined_messages, [
    int(TRAIN_PART * len(joined_messages)),
    int((TRAIN_PART + TEST_PART) * len(joined_messages)),
])

save_jsonl(train, suffix='train')
save_jsonl(test, suffix='test')
save_jsonl(valid, suffix='valid')

NameError: ignored

In [ ]:
import shutil
import subprocess
import time
import threading
from pathlib import Path


def dir_size_bytes(path):
    return int(subprocess.check_output(['du','--bytes', '--summarize', path]).split()[0].decode())

def check_chache(max_cache_size_gb=18, check_period_minutes=5):
    this_id = str(threading.get_ident())
    thread_path = Path('/threads')
    thread_path.mkdir(exist_ok=True)

    def threads_ids():
        return [str(p.name) for p in thread_path.iterdir() if p.is_file()]
    
    if not(threads_ids()):
        (thread_path / this_id).open(mode='w').close()
        while True:
            ids = threads_ids()
            if not (len(ids) == 1 and ids[0] == this_id):
                break

            print(f'Thread {this_id} is checking chache.')

            for cache_path in Path('/root/.config/Google/DriveFS').glob('**/content_cache'):
                chache_path_str = str(cache_path)
                chache_size_gb = dir_size_bytes(chache_path_str) / 10 ** 9
                if chache_size_gb > max_cache_size_gb:
                    print(f'Deleting {chache_path_str} with size {chache_size_gb} GB.')
                    shutil.rmtree(chache_path_str)
            time.sleep(check_period_minutes * 60)

False


In [12]:
# threading.Thread(target=check_chache).start()

import os

os.environ['SAVE_DIR'] = str(SAVE_DIR)
!rm --recursive --force $SAVE_DIR
!mkdir --parents $SAVE_DIR


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    task='jsonfile',
    jsonfile_datapath=str(DATA_DIR / 'data'),
    jsonfile_datatype_extension=True,

    model='transformer/generator',
    model_file=str(SAVE_DIR / 'model'),
    
    init_model='zoo:tutorial_transformer_generator/model',

    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file='zoo:tutorial_transformer_generator/model.dict',
    learn_positional_embeddings=True,
    
    lr=1e-5, optimizer='adam',
    warmup_updates=5000,
    validation_metric='ppl',
    validation_every_n_secs=60 * 60,  # running eval: valid
    save_every_n_secs=60,  # saving model checkpoint

    batchsize=12, fp16=True, fp16_impl='mem_efficient',
    
    skip_generation=True,
    
    dynamic_batching='full',

    label_turns='both',  # https://parl.ai/docs/core/teachers.html#parlai.core.teachers.ConversationTeacher
)

Thread 140129149110016 is checking chache.
10:50:57 | building data: /usr/local/lib/python3.7/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz
10:50:57 | Downloading http://parl.ai/downloads/_models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz to /usr/local/lib/python3.7/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz


10:51:55 | building dictionary first...
10:51:55 | No model with opt yet at: /content/drive/MyDrive/chatbot_model/model(.opt)
10:51:55 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,verbose: False,datapath: /usr/local/lib/python3.7/dist-packages/data,eval_dynamic_batching: None,load_from_checkpoint: True,tensorboard_logdir: None,jsonfile_datapath: /content/drive/MyDrive/chatbot_data/data,jsonfile_datatype_extension: True,label_turns: both,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,beam_block_full_context: True,beam_length_penalty: 0.65,topk: 10,topp: 0.9,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,compute_tokenized_bleu: False,interactive_mode: False,fp16_impl: mem_efficient,force_fp16_tokens: False,adafactor_eps: (1e-30, 0.001),history_reversed: False,history_add_global_end_token: None,special_tok

/usr/local/lib/python3.7/dist-packages/parlai/utils/fp16.py:487: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


10:52:17 | Overflow: setting loss scale to 32768.0
10:52:18 | Overflow: setting loss scale to 16384.0
10:52:26 | time:10s total_exs:6252 epochs:0.01
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
   .9000  4029 12081 623.5 6252             20207  11.59    .4716  4.65 6.099e-08 899.6  2697 104.6      .2877   
    total_train_updates  tpb   tps   ups  
                     30 4929 14778 2.998

10:52:36 | time:20s total_exs:10096 epochs:0.01
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss       lr  ltpb  ltps   ppl  token_acc  \
       1  5574 19834 379.9 3844             16384  13.14    .3837 4.453 1.33e-07 480.2  1709 85.92      .3137   
    total_train_updates  tpb   tps   ups  
                     66 6054 21543 3.559

10:52:46 | time:31s total_exs:12976 epochs:0.02
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss       lr  ltpb  ltps   ppl  token_acc  \
       1  6335 229

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

11:14:41 | time:1346s total_exs:77904 epochs:0.10
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
       1  6188  2334 5.092  324             16384  21.65    .3682 3.934 5.934e-06 61.33 23.13 51.13      .3342   
    total_train_updates  tpb  tps   ups  
                   2967 6250 2357 .3772

11:14:52 | time:1356s total_exs:78360 epochs:0.10
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
       1  5916 16809 44.67  456             16384  20.16    .3682 3.938 5.992e-06 75.79 215.3 51.32      .3453   
    total_train_updates  tpb   tps   ups  
                   2996 5992 17024 2.841

11:15:02 | time:1366s total_exs:78788 epochs:0.10
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
       1  6179 18472 42.65  428             16384  21.11    .4032 3.777 6.052e-06  61.7 184.4 43.69      .3603  

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


11:15:47 | time:1411s total_exs:80372 epochs:0.11
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
       1  5942 11125 25.68  384             16384  20.82    .3682 3.963 6.288e-06 65.75 123.1 52.62      .3542   
    total_train_updates  tpb   tps   ups  
                   3144 6008 11249 1.872

Thread 140129149110016 is checking chache.
Deleting /root/.config/Google/DriveFS/107380112206456973130/content_cache with size 22.315599233 GB.
11:15:57 | time:1421s total_exs:80736 epochs:0.11
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
       1  6162 17850 36.36  364             16384  21.55    .3682 3.813 6.346e-06 57.72 167.2 45.29      .3495   
    total_train_updates  tpb   tps   ups  
                   3173 6220 18017 2.897

11:16:07 | time:1432s total_exs:81132 epochs:0.11
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        l

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


FileNotFoundError: ignored

In [13]:
!ls /content/drive/MyDrive/chatbot_model/

model.checkpoint	      model.checkpoint.dict.opt
model.checkpoint.dict	      model.checkpoint.opt
model.checkpoint.dict.codecs  model.checkpoint.trainstats
